In [ ]:
import mitreattack.attackToExcel.attackToExcel as attackToExcel
import mitreattack.attackToExcel.stixToDf as stixToDf
import pandas as pd

# download and parse ATT&CK STIX data
attackdata = attackToExcel.get_stix_data("enterprise-attack")
# techniques_data = stixToDf.techniquesToDf(attackdata, "enterprise-attack")
groupsdata = stixToDf.groupsToDf(attackdata)
# softwareToDf = stixToDf.softwareToDf(attackdata)
# relationships = stixToDf.relationshipsToDf(attackdata)
# cs = stixToDf.campaignsToDf(attackdata)

# rdf = relationships['relationships']
# cdf = cs['campaigns']
# rdf = rdf.loc[rdf['source type'] == 'campaign']
# rdf = rdf.loc[rdf['target type'] == 'group']
# display(cdf)
# display(rdf)


In [ ]:

df = groupsdata['groups']
df.drop(columns=['contributors','description', 'version', 'domain', 'STIX ID', 'associated groups citations', 'relationship citations'], inplace=True)
df.rename(columns={"ID":'id', "created":"first_seen", "last modified": "last_seen"})
df.rename(columns={"ID":'id', "created":"first_seen", "last modified": "last_seen"}, inplace=True)
df = df[['id', 'name', 'associated groups', 'url', 'first_seen', 'last_seen']]
df = df.reset_index().drop(columns=['index'])
df.sort_values(by='name', inplace=True, key=lambda col: col.str.lower())
df['first_seen'] = pd.to_datetime(df['first_seen'], format='%d %B %Y')
df['last_seen'] = pd.to_datetime(df['last_seen'], format='%d %B %Y')

# Format the dates to 'yyyy_mm_dd'
df['first_seen'] = df['first_seen'].dt.strftime('%Y-%m-%d')
df['last_seen'] = df['last_seen'].dt.strftime('%Y-%m-%d')

df = df.reset_index().drop(columns=['index'])



In [16]:

# display(df)
# df.to_csv('threat_actor_groups_aliases.csv', index=False) 

,id,name,associated groups,url,first_seen,last_seen
0,G0018,admin@338,,https://attack.mitre.org/groups/G0018,2017-05-31,2020-03-18
1,G0130,Ajax Security Team,"AjaxTM, Flying Kitten, Operation Saffron Rose,...",https://attack.mitre.org/groups/G0130,2021-04-14,2023-10-09
2,G1024,Akira,"GOLD SAHARA, PUNK SPIDER",https://attack.mitre.org/groups/G1024,2024-02-20,2024-04-08
3,G0138,Andariel,"Onyx Sleet, PLUTONIUM, Silent Chollima",https://attack.mitre.org/groups/G0138,2021-09-29,2024-01-08
4,G1007,Aoqin Dragon,,https://attack.mitre.org/groups/G1007,2022-07-14,2022-10-24
...,...,...,...,...,...,...
143,G0112,Windshift,Bahamut,https://attack.mitre.org/groups/G0112,2020-06-25,2021-04-26
144,G0044,Winnti Group,Blackfly,https://attack.mitre.org/groups/G0044,2017-05-31,2023-03-20
145,G0090,WIRTE,,https://attack.mitre.org/groups/G0090,2019-05-24,2022-04-15
146,G0102,Wizard Spider,"DEV-0193, FIN12, GOLD BLACKBURN, Grim Spider, ...",https://attack.mitre.org/groups/G0102,2020-05-12,2024-04-03


In [2]:


import pandas as pd
df = pd.read_csv('../data/threat_actor_groups_aliases.csv')
gdf = pd.read_csv('../data/cyber_events.csv')


# df_expanded = df.assign(associated_groups=df['associated groups'].str.split(',') + [df['name']]).explode('associated_groups')
# df_expanded = df.assign(
#     associated_groups=df.apply(lambda x: x['associated groups'].split(',') + [x['name']], axis=1)
# ).explode('associated_groups')
df_expanded = df.assign(
    associated_groups=df.apply(lambda x: x['associated groups'].split(',') + [x['name']] if isinstance(x['associated groups'], str) else [x['name']], axis=1)
).explode('associated_groups')



# Strip leading/trailing whitespace from the new rows
df_expanded['associated_groups'] = df_expanded['associated_groups'].str.strip()

# Save or display the new dataframe with expanded rows

# If you want to print the result:
# display(df_expanded)

In [11]:

df = df_expanded
# Convert to lowercase and strip whitespace for comparison
gdf['actor'] = gdf['actor'].fillna('').str.lower().str.strip()
df['associated_groups'] = df['associated_groups'].fillna('').str.lower().str.strip()

# Define a function to find the matching 'name' from df based on 'associated_groups'
def find_matching_name(aff, df):
    matched_row = df[df['associated_groups'].apply(lambda group: aff in group)]
    if not matched_row.empty:
        return matched_row['name'].values[0]  # Return the corresponding 'name'
    return None  # Return None if no match is found

# Apply the function to replace actor with matching 'name', and filter out None values
gdf['actor'] = gdf['actor'].apply(lambda aff: find_matching_name(aff, df))

# Keep only rows where a match was found (i.e., 'actor' is not None)
gdf = gdf.dropna(subset=['actor'])

display(gdf)

# display(gdf.groupby('actor').count())


,event_date,year,month,actor,actor_type,organization,industry,motive,event_type,event_subtype,description,source_url,country,actor_country,matches
8,12/28/2023,2023,12,Tonto Team,Criminal,Bell Group,Real Estate and Rental and Leasing,Financial,Mixed,Exploitation of Application Server,The Cactus ransomware gang claims responsibili...,https://twitter.com/FalconFeedsio/status/17407...,United Kingdom of Great Britain and Northern I...,Undetermined,True
9,12/28/2023,2023,12,Tonto Team,Criminal,Tridon Australia,Manufacturing,Financial,Mixed,Exploitation of Application Server,The Cactus ransomware gang claims responsibili...,https://twitter.com/FalconFeedsio/status/17407...,Australia,Undetermined,True
23,12/22/2023,2023,12,Tonto Team,Criminal,Coop Sverige,Retail Trade,Financial,Mixed,Exploitation of Application Server,"Coop Sverige is hit with a ransomware attack, ...",https://therecord.media/coop-varmland-sweden-s...,Sweden,Undetermined,True
47,12/19/2023,2023,12,Inception,Nation-State,Russian agro-industrial in Russia,Accommodation and Food Services,Political-Espionage,Exploitive,Undetermined,Researchers from F.A.C.C.T. discover a new cam...,https://therecord.media/cloud-atlas-targets-ru...,Russian Federation,Undetermined,True
48,12/19/2023,2023,12,Inception,Nation-State,State-owned research company in Russia,"Professional, Scientific, and Technical Services",Political-Espionage,Exploitive,Undetermined,Researchers from F.A.C.C.T. discover a new cam...,https://therecord.media/cloud-atlas-targets-ru...,Russian Federation,Undetermined,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13665,1/22/2024,2024,1,Fox Kitten,Criminal,Trello,"Professional, Scientific, and Technical Services",Financial,Exploitive,Exploitation of Network Infrastructure,A threat actor with the moniker of 'emo' leaks...,https://www.bleepingcomputer.com/news/security...,Australia,Undetermined,True
13670,1/20/2024,2024,1,Akira,Criminal,Tietoevry,"Professional, Scientific, and Technical Services",Financial,Disruptive,Data Attack,Finnish IT services and enterprise cloud hosti...,https://www.bleepingcomputer.com/news/security...,Finland,Undetermined,True
13696,1/17/2024,2024,1,Tonto Team,Criminal,Schneider Electric,Manufacturing,Financial,Mixed,Data Attack; Exploitation of Application Server,Schneider Electric suffers a Cactus ransomware...,https://www.bleepingcomputer.com/news/security...,France,Undetermined,True
13724,1/11/2024,2024,1,Akira,Criminal,Lush,Wholesale Trade,Financial,Mixed,Data Attack; Exploitation of Application Server,"Lush, the privately-owned British cosmetics re...",https://therecord.media/british-cosmetics-lush...,United Kingdom of Great Britain and Northern I...,Undetermined,True


In [14]:
df = df_expanded
# Convert to lowercase and strip whitespace for comparison
gdf['actor'] = gdf['actor'].fillna('').str.lower().str.strip()
df['associated_groups'] = df['associated_groups'].fillna('').str.lower().str.strip()

# Check if any affiliation is in associated_groups and create a boolean column 'matches'
gdf['matches'] = gdf['actor'].apply(
    lambda aff: any(aff in group for group in df['associated_groups'])
)

gdf = gdf.loc[gdf['matches'] == True]

# display(gdf.groupby('motive').count())
# display(gdf.groupby('actor_type').count())
display(gdf.groupby('industry').count())


# act = 'apt28'

# display(gdf)

# Save or display the result
# gdf.to_csv('affiliations_matches.csv', index=False)

# If you want to print the result:
# print(gdf[['actor', 'matches']])

,event_date,year,month,actor,organization,industry,motive,event_type,event_subtype,description,source_url,country,actor_country,matches
actor_type,,,,,,,,,,,,,,
Criminal,376,376,376,376,376,376,376,376,375,376,376,376,375,376
Hacktivist,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Nation-State,66,66,66,66,66,66,66,66,66,66,66,66,66,66
Terrorist,16,16,16,16,16,16,16,16,16,16,16,16,16,16
Undetermined,24,24,24,24,24,24,24,24,24,24,24,24,24,24
